In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [93]:
# wine = pd.read_csv('./wine-reviews/cleaned-data-with-tasters.csv',index_col=0)
# wine = pd.read_csv('./wine-reviews/winemag-data_first150k.csv',index_col=0)
wine = pd.read_csv('./wine-reviews/winemag-data-130k-v2.csv',index_col=0)

# wine = pd.read_csv('winemag-data-130k-v5.csv',index_col=0)
print("Number of rows and columns:",wine.shape)
wine.head()

Number of rows and columns: (129971, 13)


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [ ]:
# wine['taster_name'] == ''

In [94]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [95]:
wine1 = wine.copy()
wine1.columns
# wine1['province'].unique().shape

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')

In [199]:
def recommender(x,y,z="points"):

    # Lets choice rating of wine is points, title as user_id, and variety,
    col = [x,y,'points']
    wine1 = wine[col]
    wine1 = wine1.dropna(axis=0)
    wine1 = wine1.drop_duplicates([x,y])
    wine1 = wine1[wine1['points'] >85]
    wine_pivot = wine1.pivot(index= x, columns=y,values='points').fillna(0)
    wine_pivot_matrix = csr_matrix(wine_pivot)
    # print(wine_pivot_matrix)
    knn = NearestNeighbors(n_neighbors=10,algorithm= 'brute', metric= 'cosine')
    model_knn = knn.fit(wine_pivot_matrix)
    return model_knn, wine_pivot

In [229]:
model_knn, wine_pivot = recommender(y='taster_name', x='variety')
print(wine_pivot.shape)

(670, 19)


In [230]:
# # Lets choice rating of wine is points, title as user_id, and variety,
# col = ['province','variety','points']
# wine1 = wine[col]
# wine1 = wine1.dropna(axis=0)
# wine1 = wine1.drop_duplicates(['province','variety'])
# wine1 = wine1[wine1['points'] >85]
# wine_pivot = wine1.pivot(index= 'variety',columns='province',values='points').fillna(0)
# wine_pivot_matrix = csr_matrix(wine_pivot)
# # print(wine_pivot_matrix)
# knn = NearestNeighbors(n_neighbors=10,algorithm= 'brute', metric= 'cosine')
# model_knn = knn.fit(wine_pivot_matrix)
# knn = NearestNeighbors(n_neighbors=10,algorithm= 'brute', metric= 'cosine')
# model_knn = knn.fit(wine_pivot_matrix)

In [234]:
query_index = np.random.choice(wine_pivot.shape[0])
print(query_index)
print(wine_pivot.iloc[query_index,:].shape)
distance, indice = model_knn.kneighbors(wine_pivot.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)
print(distance)
print(indice)


for i in range(0, len(distance.flatten())):
    if  i == 0:
        print('Recmmendation for {0}:\n'.format(wine_pivot.index[query_index]))
#         print('Recmmendation for {0}:\n'.format(wine_pivot.iloc[:,query_index]))
    else:
        print('{0}: {1} with distance: {2}'.format(i,wine_pivot.index[indice.flatten()[i]],distance.flatten()[i]))

519
(19,)
[[0. 0. 0. 0. 0. 0.]]
[[286 152 475 287 197 463]]
Recmmendation for Siria:

1: Ekigaïna with distance: 0.0
2: Sacy with distance: 0.0
3: Maria Gomes-Bical with distance: 0.0
4: Gouveio with distance: 0.0
5: Rosenmuskateller with distance: 0.0
